In [157]:
import pandas as pd
import numpy as np
import warnings
import utils

OAK = 'FOAK'
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

# Load and filter NREL data

In [158]:

nrel_data_file = './input_data/industry_heat_demand_characterization_nrel.csv'
nrel_data = pd.read_csv(nrel_data_file, index_col=0, encoding='windows-1252',header=0)
# Only year 2015
nrel_data = nrel_data[nrel_data['REPORTING_YEAR']==2015]
nrel_data.drop(columns=['REPORTING_YEAR'], inplace=True)
len(nrel_data)

15412

In [159]:
# Drop null demand and convert total from TJ to MJ 
nrel_data.drop(nrel_data.index[(nrel_data["Total"] ==0)],axis=0,inplace=True)
nrel_data['Heat demand (MJ/h)'] = nrel_data.apply(lambda row:row['Total']*1e6/8760, axis=1)
nrel_data.drop(columns=['Total'], inplace=True)
len(nrel_data)

13415

In [160]:
ng_df = nrel_data[nrel_data['FUEL_TYPE'].isin(['Natural Gas (Weighted U.S. Average)', 'Mixed (Industrial sector)'])]
ng_df.drop(columns=['FUEL_TYPE'], inplace=True)
len(ng_df)

7561

In [161]:
NON_AUTHORIZED_FUEL_TYPE_BLEND = ['Process Off-gas', 'Scrap Fat (from W-4 Tank)', 'Nitrile Pitch (from W-3 Tank)','Biogenic Process Derived Fuel (Glidfuel)',
                            'Biogenic Process Derived Fuel', 'Sweet Gas  Return', 'Biogeninc Process Derived Fuel (PDF)',
                             'Biogenic Process Derived Fuel (PDF)','FurnGas', 'Fuel Gas']
ng_df = ng_df[~(ng_df['FUEL_TYPE_BLEND'].isin(NON_AUTHORIZED_FUEL_TYPE_BLEND))]
ng_df.drop(columns=['FUEL_TYPE_BLEND'], inplace=True)
len(ng_df)

7561

# Compute equivalent H2 demand

In [162]:
def compute_h2_demand(heat, temp, AHF_coeffs = [0, -0.00038, .90556]):
  """Computes the equivalent hydrogen demand to produce heat 
  Args:
    - heat (float): heat demand in MJ/h
    - temp (float): temperature in degree Celsius
    - AHF_coeffs (list(float)): coefficients for Available Heat Fraction calculation
  Returns:
    - h2_demand (float): hydrogen demand in kg/h
  """
  AHF = AHF_coeffs[0]*(int(temp)^2) + AHF_coeffs[1]*int(temp) + AHF_coeffs[2]
  h2_demand = heat/(utils.h2_hhv*AHF)
  return h2_demand

In [163]:
ng_df['H2 Dem. (kg/h)'] = ng_df.apply(lambda x : compute_h2_demand(x['Heat demand (MJ/h)'], x['Temp_degC']), axis=1)

In [164]:
ng_df.columns

Index(['CITY', 'COUNTY', 'COUNTY_FIPS', 'Coal', 'Diesel', 'END_USE',
       'FACILITY_ID', 'FINAL_NAICS_CODE', 'FUEL_TYPE_OTHER', 'LPG_NGL',
       'MECS_NAICS', 'Natural_gas', 'Other', 'Process_byp', 'Pulp_Paper',
       'Residual_fuel_oil', 'STATE', 'Temp_degC', 'UNIT_NAME', 'UNIT_TYPE',
       'for_EU_sum', 'Temp_Band', 'Biogenic', 'MMTCO2E', 'Heat demand (MJ/h)',
       'H2 Dem. (kg/h)'],
      dtype='object')

In [165]:
ng_df = ng_df[['FACILITY_ID', 'STATE', 'UNIT_NAME', 'Temp_degC', 'MMTCO2E', 'Heat demand (MJ/h)', 'H2 Dem. (kg/h)']]

# Combine units at same facility with same temperature

In [170]:
ng_df_comb = ng_df.groupby(['FACILITY_ID', 'STATE', 'Temp_degC']).sum(numeric_only=True)
ng_df_comb.reset_index(inplace=True)
ng_df_comb

,FACILITY_ID,STATE,Temp_degC,MMTCO2E,Heat demand (MJ/h),H2 Dem. (kg/h)
0,1000022,TX,260.0,0.021183,48033.889338,419.645158
1,1000022,TX,900.0,0.007143,16197.474312,202.575052
2,1000025,LA,220.0,0.018848,42738.087286,366.474042
3,1000025,LA,600.0,0.031303,70981.089024,738.369449
4,1000029,WA,180.0,0.005027,11399.424699,95.973931
...,...,...,...,...,...,...
2435,1012019,IA,800.0,0.006136,13913.842628,163.022278
2436,1012036,NE,233.0,0.007181,16282.802564,140.467318
2437,1012036,NE,450.0,0.015817,35866.173214,344.141135
2438,1012089,CA,233.0,0.020767,47090.128358,406.233754


# Analyze costs and revenues with each ANR design

## Import ANR-H2 input parameters

In [167]:
h2_techs = pd.read_excel('./h2_tech.xlsx', sheet_name = 'Summary')
h2_techs.rename(columns={'CAPEX ($/MWe)':'H2 CAPEX ($/MWe)',
                 'FOM ($/MWe-year)':'H2 FOM ($/MWe-year)',
                 'VOM ($/MWhe)':'H2 VOM ($/MWhe)'}, inplace=True)

In [168]:
anrs = pd.read_excel('./ANRs.xlsx', sheet_name=OAK)
anrs.rename(columns={'CAPEX $/MWe':'ANR CAPEX ($/MWe)',
                 'FOPEX $/MWe-y':'ANR FOM ($/MWe-year)',
                 'VOM in $/MWh-e':'ANR VOM ($/MWhe)', 
                 'Startupfixedcost in $':'Start Cost ($)'}, inplace=True)
anrs

,Reactor,Type,Power in MWe,Power in MWt,MSL in MWe,Thermal Efficiency,Thermal transfer efficiency,MDT in hours,Ramp Rate (fraction of capacity/hr),Ramp Rate (MW/hr),Outlet Temp (C),ANR CAPEX ($/MWe),ANR FOM ($/MWe-year),Max Modules,ANR VOM ($/MWhe),Start Cost ($),Life (y)
0,iPWR,PWR,77.0,250,15.4,0.31,1.000,20,0.4,24.0,302,5535000,115000,12,9.75,38500,60
1,HTGR,HTGR,164.0,350,32.8,0.47,0.897,20,6.0,984.0,950,7500000,164000,4,13.00,82000,60
2,PBR-HTGR,HTGR,80.0,200,32.0,0.40,0.751,20,2.4,48.0,750,4569000,100000,12,13.00,40000,60
3,iMSR,MSR,141.0,300,28.2,0.47,0.936,20,0.6,84.6,700,4091000,85000,12,11.50,70500,60
4,Micro,HTGR,6.7,20,2.7,0.33,0.897,20,6.0,24.0,630,10902000,264000,12,0.00,3350,20


In [169]:
techs = pd.merge(h2_techs, anrs, left_on='ANR', right_on='Reactor')
techs

,Technology,ANR,Carbon intensity ANR (kgCO2eq/MWhe),H2Cap (MWe),H2Cap (kgh2/h),H2ElecCons (MWhe/kgh2),H2HeatCons (MWht/kgh2),Eq tot H2ElecCons (MWhe/kgh2),Eq tot H2HeatCons (MWht/kgh2),H2 CAPEX ($/MWe),...,MDT in hours,Ramp Rate (fraction of capacity/hr),Ramp Rate (MW/hr),Outlet Temp (C),ANR CAPEX ($/MWe),ANR FOM ($/MWe-year),Max Modules,ANR VOM ($/MWhe),Start Cost ($),Life (y)_y
0,HTSE,iPWR,12,1.3754,41.666667,0.03048,0.00816,0.033010,0.106483,646487,...,20,0.4,24.0,302,5535000,115000,12,9.75,38500,60
1,PEM,iPWR,12,1.0000,9.900990,0.10100,0.00000,0.101000,0.325806,1500000,...,20,0.4,24.0,302,5535000,115000,12,9.75,38500,60
2,Alkaline,iPWR,12,2.0000,37.037037,0.05400,0.00000,0.054000,0.174194,743865,...,20,0.4,24.0,302,5535000,115000,12,9.75,38500,60
3,HTSE,HTGR,12,0.9510,41.666667,0.00816,0.03120,0.022824,0.048562,646487,...,20,6.0,984.0,950,7500000,164000,4,13.00,82000,60
4,PEM,HTGR,12,1.0000,9.900990,0.10100,0.00000,0.101000,0.214894,1500000,...,20,6.0,984.0,950,7500000,164000,4,13.00,82000,60
5,Alkaline,HTGR,12,2.0000,37.037037,0.05400,0.00000,0.054000,0.114894,743865,...,20,6.0,984.0,950,7500000,164000,4,13.00,82000,60
6,HTSE,PBR-HTGR,12,0.8600,41.666667,0.00816,0.03120,0.020640,0.051600,646487,...,20,2.4,48.0,750,4569000,100000,12,13.00,40000,60
7,PEM,PBR-HTGR,12,1.0000,9.900990,0.10100,0.00000,0.101000,0.252500,1500000,...,20,2.4,48.0,750,4569000,100000,12,13.00,40000,60
8,Alkaline,PBR-HTGR,12,2.0000,37.037037,0.05400,0.00000,0.054000,0.135000,743865,...,20,2.4,48.0,750,4569000,100000,12,13.00,40000,60
9,HTSE,iMSR,12,0.8922,41.666667,0.00720,0.03024,0.021413,0.045559,646487,...,20,0.6,84.6,700,4091000,85000,12,11.50,70500,60


# Combine

In [171]:
ff = ng_df_comb.merge(techs, how='cross') # full facilities

ff.columns



Index(['FACILITY_ID', 'STATE', 'Temp_degC', 'MMTCO2E', 'Heat demand (MJ/h)',
       'H2 Dem. (kg/h)', 'Technology', 'ANR',
       'Carbon intensity ANR (kgCO2eq/MWhe)', 'H2Cap (MWe)', 'H2Cap (kgh2/h)',
       'H2ElecCons (MWhe/kgh2)', 'H2HeatCons (MWht/kgh2)',
       'Eq tot H2ElecCons (MWhe/kgh2)', 'Eq tot H2HeatCons (MWht/kgh2)',
       'H2 CAPEX ($/MWe)', 'H2 FOM ($/MWe-year)', 'H2 VOM ($/MWhe)',
       'Life (y)_x', 'ANR Th Eff', 'Carbon intensity (kgCO2eq/kgH2)',
       'Reactor', 'Type', 'Power in MWe', 'Power in MWt', 'MSL in MWe',
       'Thermal Efficiency', 'Thermal transfer efficiency', 'MDT in hours',
       'Ramp Rate (fraction of capacity/hr)', 'Ramp Rate (MW/hr)',
       'Outlet Temp (C)', 'ANR CAPEX ($/MWe)', 'ANR FOM ($/MWe-year)',
       'Max Modules', 'ANR VOM ($/MWhe)', 'Start Cost ($)', 'Life (y)_y'],
      dtype='object')

# Cashflows Calculations

In [172]:
import math
ff['H2 Modules'] = ff.apply(lambda x: math.ceil(x['H2 Dem. (kg/h)']/x['H2Cap (kgh2/h)']), axis=1)
ff['Depl. H2 Cap. (kgh2/h)'] = ff['H2 Modules']*ff['H2Cap (kgh2/h)']
ff['Depl. H2 Cap. (MWe)'] = ff['H2 Modules']*ff['H2Cap (MWe)']
ff['ANR Modules'] = ff.apply(lambda x: math.ceil(x['Depl. H2 Cap. (MWe)']/x['Power in MWe']), axis=1)
ff['Depl. ANR Cap. (MWe)'] = ff['ANR Modules']*ff['Power in MWe']
ff['Depl. ANR Cap. (MWt)'] = ff['ANR Modules']*ff['Power in MWt']
ff['Surplus ANR Cap. (MWe)'] = ff['Depl. ANR Cap. (MWe)']-ff['Depl. H2 Cap. (MWe)']
ff['Surplus ANR Cap. (MWt)'] = ff['Surplus ANR Cap. (MWe)']/ff['Thermal Efficiency']

In [173]:
# Compute costs in $/year
# Capital recovery factor
IR = utils.WACC
ff['ANR CRF'] = (IR/(1-((1+IR)**(-1*(ff['Life (y)_y'])))))
ff['H2 CRF'] = (IR/(1-((1+IR)**(-1*(ff['Life (y)_x'])))))

## CAPEX
ff['Tot ANR CAPEX'] = ff['Depl. ANR Cap. (MWe)']*ff['ANR CAPEX ($/MWe)']
ff['Annual ANR CAPEX'] = ff['Tot ANR CAPEX']*ff['ANR CRF']
ff['Tot H2 CAPEX'] = ff['Depl. H2 Cap. (MWe)']*ff['H2 CAPEX ($/MWe)']
ff['Annual H2 CAPEX'] = ff['Tot H2 CAPEX']*ff['H2 CRF']
ff['Annual ANR-H2 CAPEX'] = ff['Annual ANR CAPEX']+ff['Annual H2 CAPEX']

## FOM
ff['ANR FOM'] = ff['Depl. ANR Cap. (MWe)']*ff['ANR FOM ($/MWe-year)']
ff['H2 FOM'] = ff['Depl. H2 Cap. (MWe)']*ff['H2 FOM ($/MWe-year)']
ff['ANR-H2 FOM'] = ff['ANR FOM']+ff['H2 FOM']

## VOM
ff['ANR VOM'] =ff['H2 Dem. (kg/h)']*8760*ff['Eq tot H2ElecCons (MWhe/kgh2)']*ff['ANR VOM ($/MWhe)']
ff['H2 VOM'] =ff['H2 Dem. (kg/h)']*8760*ff['H2 VOM ($/MWhe)']
ff['ANR-H2 VOM'] = ff['ANR VOM']+ff['H2 VOM']

# Total cost
ff['ANR-H2 Total Cost ($/year)'] = ff['Annual ANR-H2 CAPEX'] + ff['ANR-H2 FOM'] + ff['ANR-H2 VOM']


In [174]:
# Compute revenues in $/year
# Avoided NG costs
def compute_ng_multiplier(temp, AHF_coeffs=[0, -.00038, 0.90556]):
  # Available Heat Fraction
  AHF = AHF_coeffs[0]*(int(temp)**2)+ AHF_coeffs[1]*int(temp) + AHF_coeffs[2]
  multiplier = AHF/utils.mmbtu_to_mj
  return multiplier
ff['NG Mult (MMBtu/MJ)'] = ff.apply(lambda x: compute_ng_multiplier(x['Temp_degC']), axis=1) 
# Merge state-level prices
ng_prices = pd.read_excel('./input_data/eia_aeo_industrial_sector_ng_prices.xlsx', sheet_name='state_prices')
# 2024 prices
ng_prices = ng_prices[ng_prices.year == 2024]
ng_prices.rename(columns={'price 2020USD/MMBtu':'NG price ($/MMBtu)'}, inplace=True)
ff = ff.merge(ng_prices, left_on='STATE', right_on='state')
ff['Avoided NG Cost'] = ff['NG price ($/MMBtu)']*ff['NG Mult (MMBtu/MJ)']*ff['Heat demand (MJ/h)']*8760

# H2 PTC
ff['H2 PTC'] = utils.h2_ptc*ff['H2 Dem. (kg/h)']*8760

In [175]:
ff['ANR-H2 Net Ann. Rev. ($/year)'] = -ff['ANR-H2 Total Cost ($/year)']+ff['H2 PTC']+ff['Avoided NG Cost']

In [176]:
ff.head(20)

,FACILITY_ID,STATE,Temp_degC,MMTCO2E,Heat demand (MJ/h),H2 Dem. (kg/h),Technology,ANR,Carbon intensity ANR (kgCO2eq/MWhe),H2Cap (MWe),...,H2 VOM,ANR-H2 VOM,ANR-H2 Total Cost ($/year),NG Mult (MMBtu/MJ),state,year,NG price ($/MMBtu),Avoided NG Cost,H2 PTC,ANR-H2 Net Ann. Rev. ($/year)
0,1000022,TX,260.0,0.021183,48033.889338,419.645158,HTSE,iPWR,12,1.3754,...,1.014601e+07,1.132914e+07,5.482134e+07,0.000765,TX,2024,4.096007,1.317892e+06,1.102827e+07,-4.247517e+07
1,1000022,TX,260.0,0.021183,48033.889338,419.645158,PEM,iPWR,12,1.0000,...,4.778919e+06,8.398950e+06,5.743234e+07,0.000765,TX,2024,4.096007,1.317892e+06,1.102827e+07,-4.508618e+07
2,1000022,TX,260.0,0.021183,48033.889338,419.645158,Alkaline,iPWR,12,2.0000,...,0.000000e+00,1.935462e+06,4.721338e+07,0.000765,TX,2024,4.096007,1.317892e+06,1.102827e+07,-3.486721e+07
3,1000022,TX,260.0,0.021183,48033.889338,419.645158,HTSE,HTGR,12,0.9510,...,1.014601e+07,1.123675e+07,1.349517e+08,0.000765,TX,2024,4.096007,1.317892e+06,1.102827e+07,-1.226055e+08
4,1000022,TX,260.0,0.021183,48033.889338,419.645158,PEM,HTGR,12,1.0000,...,4.778919e+06,9.605627e+06,1.393038e+08,0.000765,TX,2024,4.096007,1.317892e+06,1.102827e+07,-1.269576e+08
5,1000022,TX,260.0,0.021183,48033.889338,419.645158,Alkaline,HTGR,12,2.0000,...,0.000000e+00,2.580616e+06,1.285233e+08,0.000765,TX,2024,4.096007,1.317892e+06,1.102827e+07,-1.161772e+08
6,1000022,TX,260.0,0.021183,48033.889338,419.645158,HTSE,PBR-HTGR,12,0.8600,...,1.014601e+07,1.113238e+07,4.850557e+07,0.000765,TX,2024,4.096007,1.317892e+06,1.102827e+07,-3.615940e+07
7,1000022,TX,260.0,0.021183,48033.889338,419.645158,PEM,PBR-HTGR,12,1.0000,...,4.778919e+06,9.605627e+06,5.305688e+07,0.000765,TX,2024,4.096007,1.317892e+06,1.102827e+07,-4.071071e+07
8,1000022,TX,260.0,0.021183,48033.889338,419.645158,Alkaline,PBR-HTGR,12,2.0000,...,0.000000e+00,2.580616e+06,4.227640e+07,0.000765,TX,2024,4.096007,1.317892e+06,1.102827e+07,-2.993023e+07
9,1000022,TX,260.0,0.021183,48033.889338,419.645158,HTSE,iMSR,12,0.8922,...,1.014601e+07,1.105124e+07,6.890603e+07,0.000765,TX,2024,4.096007,1.317892e+06,1.102827e+07,-5.655987e+07


# Comparison and selection

## Best hydrogen technology

In [177]:
ff.columns

Index(['FACILITY_ID', 'STATE', 'Temp_degC', 'MMTCO2E', 'Heat demand (MJ/h)',
       'H2 Dem. (kg/h)', 'Technology', 'ANR',
       'Carbon intensity ANR (kgCO2eq/MWhe)', 'H2Cap (MWe)', 'H2Cap (kgh2/h)',
       'H2ElecCons (MWhe/kgh2)', 'H2HeatCons (MWht/kgh2)',
       'Eq tot H2ElecCons (MWhe/kgh2)', 'Eq tot H2HeatCons (MWht/kgh2)',
       'H2 CAPEX ($/MWe)', 'H2 FOM ($/MWe-year)', 'H2 VOM ($/MWhe)',
       'Life (y)_x', 'ANR Th Eff', 'Carbon intensity (kgCO2eq/kgH2)',
       'Reactor', 'Type', 'Power in MWe', 'Power in MWt', 'MSL in MWe',
       'Thermal Efficiency', 'Thermal transfer efficiency', 'MDT in hours',
       'Ramp Rate (fraction of capacity/hr)', 'Ramp Rate (MW/hr)',
       'Outlet Temp (C)', 'ANR CAPEX ($/MWe)', 'ANR FOM ($/MWe-year)',
       'Max Modules', 'ANR VOM ($/MWhe)', 'Start Cost ($)', 'Life (y)_y',
       'H2 Modules', 'Depl. H2 Cap. (kgh2/h)', 'Depl. H2 Cap. (MWe)',
       'ANR Modules', 'Depl. ANR Cap. (MWe)', 'Depl. ANR Cap. (MWt)',
       'Surplus ANR Cap. (M

In [178]:
# Select h2 tech corresponding to the maximum net annual revenue
ff.reset_index(inplace=True)
idx = ff.groupby(['FACILITY_ID', 'Temp_degC', 'ANR'])['ANR-H2 Net Ann. Rev. ($/year)'].idxmax()
max_h2 = ff.loc[idx]
print(len(ff), len(max_h2))
ff = max_h2     

36600 12200


In [179]:
ff.tail(10)

,index,FACILITY_ID,STATE,Temp_degC,MMTCO2E,Heat demand (MJ/h),H2 Dem. (kg/h),Technology,ANR,Carbon intensity ANR (kgCO2eq/MWhe),...,H2 VOM,ANR-H2 VOM,ANR-H2 Total Cost ($/year),NG Mult (MMBtu/MJ),state,year,NG price ($/MMBtu),Avoided NG Cost,H2 PTC,ANR-H2 Net Ann. Rev. ($/year)
33635,33635,1012089,CA,233.0,0.020767,47090.128358,406.233754,Alkaline,HTGR,12,...,0.000000e+00,2.498143e+06,1.281726e+08,0.000774,CA,2024,5.680861,1.814695e+06,1.067582e+07,-1.156821e+08
33642,33642,1012089,CA,233.0,0.020767,47090.128358,406.233754,HTSE,Micro,12,...,9.821757e+06,9.821757e+06,2.858583e+07,0.000774,CA,2024,5.680861,1.814695e+06,1.067582e+07,-1.609532e+07
33638,33638,1012089,CA,233.0,0.020767,47090.128358,406.233754,Alkaline,PBR-HTGR,12,...,0.000000e+00,2.498143e+06,4.192572e+07,0.000774,CA,2024,5.680861,1.814695e+06,1.067582e+07,-2.943520e+07
33641,33641,1012089,CA,233.0,0.020767,47090.128358,406.233754,Alkaline,iMSR,12,...,0.000000e+00,2.209895e+06,6.208554e+07,0.000774,CA,2024,5.680861,1.814695e+06,1.067582e+07,-4.959502e+07
33632,33632,1012089,CA,233.0,0.020767,47090.128358,406.233754,Alkaline,iPWR,12,...,0.000000e+00,1.873607e+06,4.688332e+07,0.000774,CA,2024,5.680861,1.814695e+06,1.067582e+07,-3.439281e+07
33650,33650,1012089,CA,450.0,0.045744,103725.553005,995.261727,Alkaline,HTGR,12,...,0.000000e+00,6.120382e+06,1.360861e+08,0.000696,CA,2024,5.680861,3.593802e+06,2.615548e+07,-1.063368e+08
33657,33657,1012089,CA,450.0,0.045744,103725.553005,995.261727,HTSE,Micro,12,...,2.406304e+07,2.406304e+07,5.281921e+07,0.000696,CA,2024,5.680861,3.593802e+06,2.615548e+07,-2.306993e+07
33653,33653,1012089,CA,450.0,0.045744,103725.553005,995.261727,Alkaline,PBR-HTGR,12,...,0.000000e+00,6.120382e+06,4.983919e+07,0.000696,CA,2024,5.680861,3.593802e+06,2.615548e+07,-2.008990e+07
33656,33656,1012089,CA,450.0,0.045744,103725.553005,995.261727,Alkaline,iMSR,12,...,0.000000e+00,5.414184e+06,6.958105e+07,0.000696,CA,2024,5.680861,3.593802e+06,2.615548e+07,-3.983177e+07
33647,33647,1012089,CA,450.0,0.045744,103725.553005,995.261727,Alkaline,iPWR,12,...,0.000000e+00,4.590286e+06,5.389123e+07,0.000696,CA,2024,5.680861,3.593802e+06,2.615548e+07,-2.414195e+07


In [181]:
# Compute the maximum number of temperatures for one given facility
gp_df = ff.groupby(['FACILITY_ID', 'STATE'])
max_unique_temps = gp_df['Temp_degC'].nunique().max()
print(f"The maximum number of different temperatures reported for one facility is {max_unique_temps}.")

The maximum number of different temperatures reported for one facility is 5.


In [182]:
ff.columns

Index(['index', 'FACILITY_ID', 'STATE', 'Temp_degC', 'MMTCO2E',
       'Heat demand (MJ/h)', 'H2 Dem. (kg/h)', 'Technology', 'ANR',
       'Carbon intensity ANR (kgCO2eq/MWhe)', 'H2Cap (MWe)', 'H2Cap (kgh2/h)',
       'H2ElecCons (MWhe/kgh2)', 'H2HeatCons (MWht/kgh2)',
       'Eq tot H2ElecCons (MWhe/kgh2)', 'Eq tot H2HeatCons (MWht/kgh2)',
       'H2 CAPEX ($/MWe)', 'H2 FOM ($/MWe-year)', 'H2 VOM ($/MWhe)',
       'Life (y)_x', 'ANR Th Eff', 'Carbon intensity (kgCO2eq/kgH2)',
       'Reactor', 'Type', 'Power in MWe', 'Power in MWt', 'MSL in MWe',
       'Thermal Efficiency', 'Thermal transfer efficiency', 'MDT in hours',
       'Ramp Rate (fraction of capacity/hr)', 'Ramp Rate (MW/hr)',
       'Outlet Temp (C)', 'ANR CAPEX ($/MWe)', 'ANR FOM ($/MWe-year)',
       'Max Modules', 'ANR VOM ($/MWhe)', 'Start Cost ($)', 'Life (y)_y',
       'H2 Modules', 'Depl. H2 Cap. (kgh2/h)', 'Depl. H2 Cap. (MWe)',
       'ANR Modules', 'Depl. ANR Cap. (MWe)', 'Depl. ANR Cap. (MWt)',
       'Surplus AN

In [184]:
compdf = ff[['FACILITY_ID', 'STATE', 'ANR', 'Technology']]
compdf.drop_duplicates(inplace=True)
compdf.set_index(['FACILITY_ID', 'STATE'], inplace=True)
compdf['# Temps'] = ff.groupby(['FACILITY_ID', 'STATE'])['Temp_degC'].nunique()


In [185]:
compdf = compdf.reset_index()
compdf

,FACILITY_ID,STATE,ANR,Technology,# Temps
0,1000022,TX,HTGR,Alkaline,2
1,1000022,TX,Micro,HTSE,2
2,1000022,TX,PBR-HTGR,Alkaline,2
3,1000022,TX,iMSR,Alkaline,2
4,1000022,TX,iPWR,Alkaline,2
...,...,...,...,...,...
5651,1012089,CA,HTGR,Alkaline,2
5652,1012089,CA,Micro,HTSE,2
5653,1012089,CA,PBR-HTGR,Alkaline,2
5654,1012089,CA,iMSR,Alkaline,2


In [186]:
# All ANR-H2 for heat
compdf = compdf.set_index(['FACILITY_ID', 'STATE', 'ANR', 'Technology'])
compdf['All ANR-H2 ($/year)'] = ff.groupby(['FACILITY_ID', 'STATE', 'ANR', 'Technology'])['ANR-H2 Net Ann. Rev. ($/year)'].sum()
compdf 

# Temps  All ANR-H2 ($/year)
FACILITY_ID STATE ANR      Technology                              
1000022     TX    HTGR     Alkaline          2        -2.361223e+08
                  Micro    HTSE              2        -2.527741e+07
                  PBR-HTGR Alkaline          2        -6.362843e+07
                  iMSR     Alkaline          2        -1.040831e+08
                  iPWR     Alkaline          2        -7.383612e+07
...                                        ...                  ...
1012089     CA    HTGR     Alkaline          2        -2.220190e+08
                  Micro    HTSE              2        -3.916524e+07
                  PBR-HTGR Alkaline          2        -4.952511e+07
                  iMSR     Alkaline          2        -8.942679e+07
                  iPWR     Alkaline          2        -5.853475e+07

[5656 rows x 2 columns]

In [187]:
ff.head(3)

,index,FACILITY_ID,STATE,Temp_degC,MMTCO2E,Heat demand (MJ/h),H2 Dem. (kg/h),Technology,ANR,Carbon intensity ANR (kgCO2eq/MWhe),...,H2 VOM,ANR-H2 VOM,ANR-H2 Total Cost ($/year),NG Mult (MMBtu/MJ),state,year,NG price ($/MMBtu),Avoided NG Cost,H2 PTC,ANR-H2 Net Ann. Rev. ($/year)
5,5,1000022,TX,260.0,0.021183,48033.889338,419.645158,Alkaline,HTGR,12,...,0.000000e+00,2.580616e+06,1.285233e+08,0.000765,TX,2024,4.096007,1.317892e+06,1.102827e+07,-1.161772e+08
12,12,1000022,TX,260.0,0.021183,48033.889338,419.645158,HTSE,Micro,12,...,1.014601e+07,1.014601e+07,2.897787e+07,0.000765,TX,2024,4.096007,1.317892e+06,1.102827e+07,-1.663171e+07
8,8,1000022,TX,260.0,0.021183,48033.889338,419.645158,Alkaline,PBR-HTGR,12,...,0.000000e+00,2.580616e+06,4.227640e+07,0.000765,TX,2024,4.096007,1.317892e+06,1.102827e+07,-2.993023e+07


In [188]:
compdf = compdf.reset_index()
compdf.set_index(['FACILITY_ID', 'STATE'], inplace=True)
compdf1 = compdf[compdf['# Temps']==1]
compdf2 = compdf[compdf['# Temps']==2]
compdf3 = compdf[compdf['# Temps']==3]
compdf4 = compdf[compdf['# Temps']==4]
compdf5 = compdf[compdf['# Temps']==5]
temperatures_df = ff[['FACILITY_ID', 'STATE', 'Temp_degC']]
temperatures_df.drop_duplicates(inplace=True)
temperatures_df.sort_values(by=['FACILITY_ID', 'STATE', 'Temp_degC'], inplace=True)
temperatures_df

,FACILITY_ID,STATE,Temp_degC
5,1000022,TX,260.0
20,1000022,TX,900.0
3530,1000025,LA,220.0
3545,1000025,LA,600.0
5750,1000029,WA,180.0
...,...,...,...
31385,1012019,IA,800.0
11285,1012036,NE,233.0
11300,1012036,NE,450.0
33635,1012089,CA,233.0


In [65]:
# TODO unit_name delete
# Temp 1
compdf1['Temp1'] = temperatures_df.groupby(['FACILITY_ID', 'STATE',])['Temp_degC'].min()
compdf2['Temp1'] = temperatures_df.groupby(['FACILITY_ID', 'STATE', 'UNIT_NAME'])['Temp_degC'].min()
compdf3['Temp1'] = temperatures_df.groupby(['FACILITY_ID', 'STATE', 'UNIT_NAME'])['Temp_degC'].min()
compdf4['Temp1'] = temperatures_df.groupby(['FACILITY_ID', 'STATE', 'UNIT_NAME'])['Temp_degC'].min()
compdf5['Temp1'] = temperatures_df.groupby(['FACILITY_ID', 'STATE', 'UNIT_NAME'])['Temp_degC'].min()
# Temp 2
compdf2['Temp2'] = temperatures_df.groupby(['FACILITY_ID', 'STATE', 'UNIT_NAME'])['Temp_degC'].nsmallest(2).iloc[1]
compdf3['Temp2'] = temperatures_df.groupby(['FACILITY_ID', 'STATE', 'UNIT_NAME'])['Temp_degC'].nsmallest(3).iloc[1]
compdf4['Temp2'] = temperatures_df.groupby(['FACILITY_ID', 'STATE', 'UNIT_NAME'])['Temp_degC'].nsmallest(4).iloc[1]
compdf5['Temp2'] = temperatures_df.groupby(['FACILITY_ID', 'STATE', 'UNIT_NAME'])['Temp_degC'].nsmallest(5).iloc[1]
# Temp 3
compdf3['Temp3'] = temperatures_df.groupby(['FACILITY_ID', 'STATE', 'UNIT_NAME'])['Temp_degC'].nsmallest(3).iloc[2]
compdf4['Temp3'] = temperatures_df.groupby(['FACILITY_ID', 'STATE', 'UNIT_NAME'])['Temp_degC'].nsmallest(4).iloc[2]
compdf5['Temp3'] = temperatures_df.groupby(['FACILITY_ID', 'STATE', 'UNIT_NAME'])['Temp_degC'].nsmallest(5).iloc[2]
# temp 4 
compdf4['Temp4'] = temperatures_df.groupby(['FACILITY_ID', 'STATE', 'UNIT_NAME'])['Temp_degC'].nsmallest(4).iloc[3]
compdf5['Temp4'] = temperatures_df.groupby(['FACILITY_ID', 'STATE', 'UNIT_NAME'])['Temp_degC'].nsmallest(5).iloc[3]
# Temp 5
compdf5['Temp5'] = temperatures_df.groupby(['FACILITY_ID', 'STATE', 'UNIT_NAME'])['Temp_degC'].nsmallest(5).iloc[4]

In [66]:
compdf_new = pd.concat([compdf1, compdf2, compdf3, compdf4, compdf5])
compdf_new

ANR Technology  # Temps  \
FACILITY_ID STATE UNIT_NAME                                 
1000056     MA    GP-1           HTGR   Alkaline      1.0   
                  GP-1          Micro       HTSE      1.0   
                  GP-1       PBR-HTGR       HTSE      1.0   
                  GP-1           iMSR   Alkaline      1.0   
                  GP-1           iPWR       HTSE      1.0   
...                               ...        ...      ...   
1012019     IA    GP-01         Micro   Alkaline      5.0   
                  GP-01      PBR-HTGR   Alkaline      5.0   
                  GP-01          iMSR   Alkaline      5.0   
                  GP-01          iPWR   Alkaline      5.0   
                  GP-01         Micro       HTSE      5.0   

                             All ANR-H2 ($/year)   Temp1  Temp2  Temp3  Temp4  \
FACILITY_ID STATE UNIT_NAME                                                     
1000056     MA    GP-1             -9.984467e+07  1500.0    NaN    NaN    NaN   
                  GP-1             -4.279257e+07  1500.0    NaN    NaN    NaN   
                  GP-1             -3.848769e+07  1500.0    NaN    NaN    NaN   
                  GP-1             -3.287236e+07  1500.0    NaN    NaN    NaN   
                  GP-1             -4.679749e+07  1500.0    NaN    NaN    NaN   
...                                          ...     ...    ...    ...    ...   
1012019     IA    GP-01            -2.450617e+07   150.0  900.0  260.0  900.0   
                  GP-01            -1.754111e+08   150.0  900.0  260.0  900.0   
                  GP-01            -2.773186e+08   150.0  900.0  260.0  900.0   
                  GP-01            -2.026007e+08   150.0  900.0  260.0  900.0   
                  GP-01            -1.714629e+07   150.0  900.0  260.0  900.0   

                             Temp5  
FACILITY_ID STATE UNIT_NAME         
1000056     MA    GP-1         NaN  
                  GP-1         NaN  
                  GP-1         NaN  
                  GP-1         NaN  
                  GP-1         NaN  
...                            ...  
1012019     IA    GP-01      220.0  
                  GP-01      220.0  
                  GP-01      220.0  
                  GP-01      220.0  
                  GP-01      220.0  

[15482 rows x 9 columns]

In [ ]:
# First temperature
compdf = compdf.reset_index()
compdf.set_index(['FACILITY_ID', 'STATE', 'UNIT_NAME'], inplace=True)
temperatures_df = ff[['FACILITY_ID', 'STATE', 'UNIT_NAME', 'Temp_degC']]
temperatures_df.drop_duplicates(inplace=True)
temperatures_df.sort_values(by=['FACILITY_ID', 'STATE', 'UNIT_NAME','Temp_degC'], inplace=True)
compdf['Temp1'] =temperatures_df.groupby(['FACILITY_ID', 'STATE', 'UNIT_NAME'])['Temp_degC'].min()
compdf['Temp1 check'] =temperatures_df.groupby(['FACILITY_ID', 'STATE', 'UNIT_NAME'])['Temp_degC'].nsmallest(5).iloc[0]
compdf['Temp2'] = temperatures_df.groupby(['FACILITY_ID', 'STATE', 'UNIT_NAME'])['Temp_degC'].nsmallest(5).iloc[1]
compdf['Temp3'] = temperatures_df.groupby(['FACILITY_ID', 'STATE', 'UNIT_NAME'])['Temp_degC'].nsmallest(5).iloc[2]
compdf['Temp4'] = temperatures_df.groupby(['FACILITY_ID', 'STATE', 'UNIT_NAME'])['Temp_degC'].nsmallest(5).iloc[3]
compdf['Temp5'] = temperatures_df.groupby(['FACILITY_ID', 'STATE', 'UNIT_NAME'])['Temp_degC'].nsmallest(5).iloc[4]
compdf['Temp5 check'] = ff.groupby(['FACILITY_ID', 'STATE', 'UNIT_NAME'])['Temp_degC'].max()
compdf

ANR Technology  # Temps  \
FACILITY_ID STATE UNIT_NAME                                         
1000022     TX    GP-Blr                 HTGR   Alkaline        2   
                  GP-Blr                Micro   Alkaline        2   
                  GP-Blr             PBR-HTGR   Alkaline        2   
                  GP-Blr                 iMSR   Alkaline        2   
                  GP-Blr                 iPWR   Alkaline        2   
...                                       ...        ...      ...   
1012089     CA    GP-Multiple Units      HTGR   Alkaline        2   
                  GP-Multiple Units     Micro       HTSE        2   
                  GP-Multiple Units  PBR-HTGR   Alkaline        2   
                  GP-Multiple Units      iMSR   Alkaline        2   
                  GP-Multiple Units      iPWR   Alkaline        2   

                                     All ANR-H2 ($/year)  Temp1  Temp1 check  \
FACILITY_ID STATE UNIT_NAME                                                    
1000022     TX    GP-Blr                   -2.402888e+08  260.0        260.0   
                  GP-Blr                   -7.570804e+06  260.0        260.0   
                  GP-Blr                   -6.779491e+07  260.0        260.0   
                  GP-Blr                   -1.084453e+08  260.0        260.0   
                  GP-Blr                   -7.842678e+07  260.0        260.0   
...                                                  ...    ...          ...   
1012089     CA    GP-Multiple Units        -2.295060e+08  233.0        260.0   
                  GP-Multiple Units        -3.195600e+07  233.0        260.0   
                  GP-Multiple Units        -5.701219e+07  233.0        260.0   
                  GP-Multiple Units        -9.722582e+07  233.0        260.0   
                  GP-Multiple Units        -6.669771e+07  233.0        260.0   

                                     Temp2  Temp3  Temp4  Temp5  Temp5 check  
FACILITY_ID STATE UNIT_NAME                                                   
1000022     TX    GP-Blr             900.0  260.0  900.0  220.0        900.0  
                  GP-Blr             900.0  260.0  900.0  220.0        900.0  
                  GP-Blr             900.0  260.0  900.0  220.0        900.0  
                  GP-Blr             900.0  260.0  900.0  220.0        900.0  
                  GP-Blr             900.0  260.0  900.0  220.0        900.0  
...                                    ...    ...    ...    ...          ...  
1012089     CA    GP-Multiple Units  900.0  260.0  900.0  220.0        450.0  
                  GP-Multiple Units  900.0  260.0  900.0  220.0        450.0  
                  GP-Multiple Units  900.0  260.0  900.0  220.0        450.0  
                  GP-Multiple Units  900.0  260.0  900.0  220.0        450.0  
                  GP-Multiple Units  900.0  260.0  900.0  220.0        450.0  

[13189 rows x 11 columns]

In [ ]:
# Load direct heat results
maxdf = pd.read_csv('./input_data/direct_heat_maxv/Full_Spread_SMRs.csv')
maxdf = maxdf[['FACILITY_ID', 'STATE', 'UNIT_NAME', 'Temp_degC','Industry', 'Thermal MWh/hr', 'Emissions', 'NG_HLMP_mod',\
               'Reactor', 'Modules','SMR_Capacity', 'SMR_Capacity_e', 'Surplus_Capacity','Surplus_Capacity_e', 'Annual_Cost_FOAK',\
                'Annual_Cost_NOAK']]


In [ ]:
compdf =compdf.merge(maxdf,)

MergeError: No common columns to perform merge on. Merge options: left_on=None, right_on=None, left_index=False, right_index=False